In [1]:
import java.util.concurrent._
import language.implicitConversions
val parEngine = Executors.newFixedThreadPool(2)

scala.language$@1cad560c

In [2]:
// id : ex_7.1

class Par[A](ap : () => A) {
    
    val a : () => A = ap
        
    def get : A = a()
    
}

object Par {
    
    def unit[A](a : => A) : Par[A] = new Par(() => a)

    def map2[A,B,C](pa : Par[A], pb: Par[B])(f: (A, B) => C): Par[C] = {
        unit(f(pa.get,pb.get))
    }
    
}


$line19.$read$$iw$$iw$Par$@4b8e2c57

In [3]:
def sum(ints: IndexedSeq[Int]): Par[Int] = 
    if (ints.size <= 1) Par.unit(ints.headOption getOrElse 0) 
    else {
        val (l,r) = ints.splitAt(ints.length/2)
        Par.map2(sum(l), sum(r))(_ + _)
    }

sum: (ints: IndexedSeq[Int])Par[Int]


In [4]:
sum(IndexedSeq(1,2,3,4,5)).get

15

In [5]:
// id : ex_7.2

// Shamelessly (or shamefully) taking the relevant portions from FP in Scala answers
type Par[A] = ExecutorService => Future[A]

object Par {

  def run[A](s: ExecutorService)(a: Par[A]): Future[A] = a(s)

  /*
  `unit` is represented as a function that returns a `UnitFuture`, which is a 
  simple implementation of `Future` that just wraps a constant value. It doesn't 
  use the `ExecutorService` at all. It's always done and can't be cancelled. Its 
  `get` method simply returns the value that we gave it.
  */
  def unit[A](a: A): Par[A] = (es: ExecutorService) => UnitFuture(a) 

  private case class UnitFuture[A](get: A) extends Future[A] {
    def isDone = true
    def get(timeout: Long, units: TimeUnit) = get
    def isCancelled = false
    def cancel(evenIfRunning: Boolean): Boolean = false
  }

  /*
  `map2` doesn't evaluate the call to `f` in a separate logical thread, in accord 
  with our design choice of having `fork` be the sole function in the API for 
  controlling parallelism. We can always do `fork(map2(a,b)(f))` if we want the 
  evaluation of `f` to occur in a separate thread.
  */
  def map2[A,B,C](a: Par[A], b: Par[B])(f: (A,B) => C): Par[C] = 
    (es: ExecutorService) => {
      val af = a(es)
      val bf = b(es)
      /* 
      This implementation of `map2` does _not_ respect timeouts. It simply passes 
      the `ExecutorService` on to both `Par` values, waits for the results of the 
      Futures `af` and `bf`, applies `f` to them, and wraps them in a `UnitFuture`. 
      In order to respect timeouts, we'd need a new `Future` implementation that 
      records the amount of time spent evaluating `af`, then subtracts that time 
      from the available time allocated for evaluating `bf`.
      */
      UnitFuture(f(af.get, bf.get)) 
    }

  /*
  This is the simplest and most natural implementation of `fork`, but there are 
  some problems with it--for one, the outer `Callable` will block waiting for the 
  "inner" task to complete. Since this blocking occupies a thread in our thread pool, 
  or whatever resource backs the `ExecutorService`, this implies that we're losing 
  out on some potential parallelism. Essentially, we're using two threads when one 
  should suffice. This is a symptom of a more serious problem with the implementation, 
  and we will discuss this later in the chapter.
  */
  def fork[A](a: => Par[A]): Par[A] = 
    es => es.submit(new Callable[A] {
      def call = a(es).get
    })

  def lazyUnit[A](a: => A): Par[A] = fork(unit(a))

}


$line22.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$Par$@6437bd1c

In [6]:
// id : try ex_7.2

def sum(ints: IndexedSeq[Int]): Par[Int] = 
    if (ints.size <= 1) Par.unit(ints.headOption getOrElse 0) 
    else {
        val (l,r) = ints.splitAt(ints.length/2)
        Par.map2(sum(l), sum(r))(_ + _)
    }

val future = Par.run(Executors.newFixedThreadPool(1))(sum(IndexedSeq(1,2,3,4,5)))

println("future", future)
println("get", future.get)

(future,UnitFuture(15))
(get,15)


null

In [7]:
// id : ex_7.3
import java.util.concurrent.Future

type Par[A] = ExecutorService => Future[A]

object Par {

    def run[A](s: ExecutorService)(a: Par[A]): Future[A] = a(s)

    def unit[A](a: () => A): Par[A] = (es: ExecutorService) => es.submit(new Callable[A] {
          def call = a()
    })

    def map2[A,B,C](a: Par[A], b: Par[B])(f: (A,B) => C): Par[C] = 
        (es: ExecutorService) => {
        val af = a(es)
        val bf = b(es)
        Map2UnitFuture(af, bf, f) 
    }

    // https://github.com/fpinscala/fpinscala/blob/master/answerkey/parallelism/03.answer.scala
    
    case class Map2UnitFuture[A, B, C](
        a: Future[A], 
        b: Future[B],
        f : (A, B) => C) extends Future[C] {
        
        @volatile var cache: Option[C] = None
        def isDone = cache.isDefined
        def isCancelled = a.isCancelled || b.isCancelled
        def cancel(evenIfRunning: Boolean) =
            a.cancel(evenIfRunning) || b.cancel(evenIfRunning)
        def get = compute(Long.MaxValue)
        def get(timeout: Long, units: TimeUnit): C =
            compute(TimeUnit.NANOSECONDS.convert(timeout, units))

        private def compute(timeoutInNanos: Long): C = cache match {
            case Some(c) => c
            case None =>
              println("Timeout " + TimeUnit.SECONDS.convert(timeoutInNanos, TimeUnit.NANOSECONDS))
              val start = System.nanoTime
              val ar = a.get(timeoutInNanos, TimeUnit.NANOSECONDS)
              val stop = System.nanoTime;val aTime = stop-start
              println("Timeout " + TimeUnit.SECONDS.convert(timeoutInNanos - aTime, TimeUnit.NANOSECONDS))
              val br = b.get(timeoutInNanos - aTime, TimeUnit.NANOSECONDS)
              val ret = f(ar, br)
              cache = Some(ret)
              ret
        }
    }

}

$line24.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$Par$@3676c50c

In [8]:
def sum(ints: IndexedSeq[Int]): Par[Int] = 
    if (ints.size <= 1) {
        ints match {
            case IndexedSeq(x) => Par.unit(() => x)
            case IndexedSeq() => Par.unit(() => 0)
        }
    } else {
        val (l,r) = ints.splitAt(ints.length/2)
        Par.map2(sum(l), sum(r))({
            Thread.sleep(1000)
            _ + _
        })
    }


sum: (ints: IndexedSeq[Int])Par[Int]


In [9]:
val future = Par.run(Executors.newFixedThreadPool(4))(sum(IndexedSeq(1,2,3,4,5,6,7,8,9,10)))

Map2UnitFuture(Map2UnitFuture(Map2UnitFuture(java.util.concurrent.FutureTask@569fdc76,java.util.concurrent.FutureTask@734bda03,<function2>),Map2UnitFuture(java.util.concurrent.FutureTask@5feff8e1,Map2UnitFuture(java.util.concurrent.FutureTask@546ab144,java.util.concurrent.FutureTask@7cbeabd3,<function2>),<function2>),<function2>),Map2UnitFuture(Map2UnitFuture(java.util.concurrent.FutureTask@5108ed85,java.util.concurrent.FutureTask@71002fb5,<function2>),Map2UnitFuture(java.util.concurrent.FutureTask@70e14040,Map2UnitFuture(java.util.concurrent.FutureTask@79088dbc,java.util.concurrent.FutureTask@6ffa33dd,<function2>),<function2>),<function2>),<function2>)

In [10]:
future.get(1, TimeUnit.NANOSECONDS)

Timeout 0
Timeout 0
Timeout 0
Timeout 0
Timeout 0
Timeout 0
Timeout 0
Timeout 0
Timeout 0
Timeout 0
Timeout 0
Timeout 0
Timeout 0
Timeout 0
Timeout 0
Timeout 0
Timeout 0
Timeout 0


55

In [11]:
val sum2 = Par.map2(Par.unit(() => { Thread.sleep(10000); 1}), 
                    Par.unit(() => { Thread.sleep(10000); 2}))((a: Int, b: Int) => a + b)
sum2(Executors.newFixedThreadPool(4)).get(0, TimeUnit.NANOSECONDS)

Timeout 0


java.util.concurrent.TimeoutException: java.util.concurrent.TimeoutException

In [12]:
val sum2 = Par.map2(Par.unit(() => { Thread.sleep(10000); 1}), 
                    Par.unit(() => { Thread.sleep(30000); 2}))((a: Int, b: Int) => a + b)
sum2(Executors.newFixedThreadPool(4)).get(11, TimeUnit.SECONDS)

Timeout 11
Timeout 1


java.util.concurrent.TimeoutException: java.util.concurrent.TimeoutException

In [13]:
val sum2 = Par.map2(Par.unit(() => { Thread.sleep(2500); 1}), 
                    Par.unit(() => { Thread.sleep(2500); 2}))((a: Int, b: Int) => a + b)
Par.run(Executors.newFixedThreadPool(4))(sum2).get(3, TimeUnit.SECONDS)

Timeout 3
Timeout 0


3

In [14]:
// id : ex_7.4

type Par[A] = ExecutorService => Future[A]

object Par {

  def run[A](s: ExecutorService)(a: Par[A]): Future[A] = a(s)

  def unit[A](a: A): Par[A] = (es: ExecutorService) => UnitFuture(a) 

  private case class UnitFuture[A](get: A) extends Future[A] {
    def isDone = true
    def get(timeout: Long, units: TimeUnit) = get
    def isCancelled = false
    def cancel(evenIfRunning: Boolean): Boolean = false
  }

  def map2[A,B,C](a: Par[A], b: Par[B])(f: (A,B) => C): Par[C] = 
    (es: ExecutorService) => {
      val af = a(es)
      val bf = b(es)
      UnitFuture(f(af.get, bf.get)) 
    }

  def fork[A](a: => Par[A]): Par[A] = 
    es => es.submit(new Callable[A] {
      def call = a(es).get
    })

  def lazyUnit[A](a: => A): Par[A] = fork(unit(a))

  def asyncF[A,B](f: A => B): A => Par[B] = a => lazyUnit(f(a))
}


$line35.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$Par$@59abab4f

In [15]:
// id : try ex_7.4

val func = Par.asyncF((a : Int) => a + 1)
val future = Par.run(Executors.newSingleThreadExecutor)(func(5))
future.get

6

In [16]:
def sortPar(parList: Par[List[Int]]): Par[List[Int]] = Par.map2(parList, Par.unit(()))((a, _) => a.sorted)
def map[A,B](pa: Par[A])(f: A => B): Par[B] = Par.map2(pa, Par.unit(()))((a,_) => f(a))

sortPar: (parList: Par[List[Int]])Par[List[Int]]
map: [A, B](pa: Par[A])(f: A => B)Par[B]


In [17]:
def sortPar(parList: Par[List[Int]]) = map(parList)(_.sorted)

sortPar: (parList: Par[List[Int]])Par[List[Int]]


In [18]:
// id : ex_7.5

/* Note: Show sequence for the pattern
def sequence[A](fs: List[Rand[A]]) : Rand[List[A]] = {
    rng => {
        fs match {
            case Nil => (List().asInstanceOf[List[A]], rng)
            case r2s :: rest => {
                val (r2sa, r2sr) = r2s(rng)
                val rem = sequence(rest)(r2sr)
                (r2sa :: rem._1, rem._2)
            }
        }
    }
}
*/

// def map2[A,B,C](a : Par[A], b : Par[B])(f: (A, B) => C): Par[C] = 
//     map2       (a : Par[A], b : Par[A])(f: (A, A) => List[A]) : Par[List[A]]
def sequence[A](ps: List[Par[A]]): Par[List[A]] = ps match {
    case Nil => Par.unit(List().asInstanceOf[List[A]])
    case p :: pr   => Par.map2(p, sequence(pr))((a1: A, a2 : List[A]) => List(a1) ++ a2)
}

sequence: [A](ps: List[Par[A]])Par[List[A]]


In [19]:
// id : try ex_7.5

val listOfPars = sequence(List(Par.unit(1), Par.unit(2), Par.unit(3), Par.unit(4)))
Par.run(parEngine)(listOfPars).get

[1, 2, 3, 4]

In [20]:
def parMap[A,B](ps: List[A])(f: A => B): Par[List[B]] = Par.fork { 
    val fbs: List[Par[B]] = ps.map(Par.asyncF(f))
    sequence(fbs)
}

parMap: [A, B](ps: List[A])(f: A => B)Par[List[B]]


In [21]:
// id : ex_7.6
def parFilter[A](as: List[A])(f: A => Boolean): Par[List[A]] = 
    map(parMap(as)(x => if (f(x)) Some(x) else None))(xs => xs.flatMap(x => x))

parFilter: [A](as: List[A])(f: A => Boolean)Par[List[A]]


In [22]:
// id : try ex_7.6
val xpool7 = Executors.newFixedThreadPool(7)
Par.run(xpool7)(parFilter(List(1,2,3,4,5,6))(x => {
    println("Sleeping 1 second for", x)
    Thread.sleep(1000)
    x > 4
})).get

(Sleeping 1 second for,2)
(Sleeping 1 second for,6)
(Sleeping 1 second for,1)
(Sleeping 1 second for,5)
(Sleeping 1 second for,4)
(Sleeping 1 second for,3)


[5, 6]

In [23]:
// id : book answer ex_7.6
def parFilter[A](l: List[A])(f: A => Boolean): Par[List[A]] = {
  val pars: List[Par[List[A]]] = 
    l map (Par.asyncF((a: A) => if (f(a)) List(a) else List())) 
  map(sequence(pars))(_.flatten) // convenience method on `List` for concatenating a list of lists
}

parFilter: [A](l: List[A])(f: A => Boolean)Par[List[A]]


In [24]:
// id : try book answer ex_7.6
Par.run(xpool7)(parFilter(List(1,2,3,4,5,6))(x => {
    println("Sleeping 1 second for", x)
    Thread.sleep(1000)
    x > 4
})).get

(Sleeping 1 second for,1)
(Sleeping 1 second for,6)
(Sleeping 1 second for,5)
(Sleeping 1 second for,4)
(Sleeping 1 second for,3)
(Sleeping 1 second for,2)


[5, 6]

In [25]:
// id : actor from fpinscala

import java.util.concurrent.atomic.{AtomicInteger, AtomicReference}
import java.util.concurrent.{Callable,ExecutorService}
import annotation.tailrec

/*
 * Implementation is taken from `scalaz` library, with only minor changes. See:
 *
 * https://github.com/scalaz/scalaz/blob/scalaz-seven/concurrent/src/main/scala/scalaz/concurrent/Actor.scala
 *
 * This code is copyright Andriy Plokhotnyuk, Runar Bjarnason, and other contributors,
 * and is licensed using 3-clause BSD, see LICENSE file at:
 *
 * https://github.com/scalaz/scalaz/blob/scalaz-seven/etc/LICENCE
 */

/**
 * Processes messages of type `A`, one at a time. Messages are submitted to
 * the actor with the method `!`. Processing is typically performed asynchronously,
 * this is controlled by the provided `strategy`.
 *
 * Memory consistency guarantee: when each message is processed by the `handler`, any memory that it
 * mutates is guaranteed to be visible by the `handler` when it processes the next message, even if
 * the `strategy` runs the invocations of `handler` on separate threads. This is achieved because
 * the `Actor` reads a volatile memory location before entering its event loop, and writes to the same
 * location before suspending.
 *
 * Implementation based on non-intrusive MPSC node-based queue, described by Dmitriy Vyukov:
 * [[http://www.1024cores.net/home/lock-free-algorithms/queues/non-intrusive-mpsc-node-based-queue]]
 *
 * @see scalaz.concurrent.Promise for a use case.
 *
 * @param handler  The message handler
 * @param onError  Exception handler, called if the message handler throws any `Throwable`.
 * @param strategy Execution strategy, for example, a strategy that is backed by an `ExecutorService`
 * @tparam A       The type of messages accepted by this actor.
 */
final class Actor[A](strategy: Strategy)(handler: A => Unit, onError: Throwable => Unit = throw(_)) {
  self =>

  private val tail = new AtomicReference(new Node[A]())
  private val suspended = new AtomicInteger(1)
  private val head = new AtomicReference(tail.get)

  /** Alias for `apply` */
  def !(a: A) {
    val n = new Node(a)
    head.getAndSet(n).lazySet(n)
    trySchedule()
  }

  /** Pass the message `a` to the mailbox of this actor */
  def apply(a: A) {
    this ! a
  }

  def contramap[B](f: B => A): Actor[B] =
    new Actor[B](strategy)((b: B) => (this ! f(b)), onError)

  private def trySchedule() {
    if (suspended.compareAndSet(1, 0)) schedule()
  }

  private def schedule() {
    strategy(act())
  }

  private def act() {
    val t = tail.get
    val n = batchHandle(t, 1024)
    if (n ne t) {
      n.a = null.asInstanceOf[A]
      tail.lazySet(n)
      schedule()
    } else {
      suspended.set(1)
      if (n.get ne null) trySchedule()
    }
  }

  @tailrec
  private def batchHandle(t: Node[A], i: Int): Node[A] = {
    val n = t.get
    if (n ne null) {
      try {
        handler(n.a)
      } catch {
        case ex: Throwable => onError(ex)
      }
      if (i > 0) batchHandle(n, i - 1) else n
    } else t
  }
}

private class Node[A](var a: A = null.asInstanceOf[A]) extends AtomicReference[Node[A]]

object Actor {

  /** Create an `Actor` backed by the given `ExecutorService`. */
  def apply[A](es: ExecutorService)(handler: A => Unit, onError: Throwable => Unit = throw(_)): Actor[A] =
    new Actor(Strategy.fromExecutorService(es))(handler, onError)
}

/**
 * Provides a function for evaluating expressions, possibly asynchronously.
 * The `apply` function should typically begin evaluating its argument
 * immediately. The returned thunk can be used to block until the resulting `A`
 * is available.
 */
trait Strategy {
  def apply[A](a: => A): () => A
}

object Strategy {

  /**
   * We can create a `Strategy` from any `ExecutorService`. It's a little more
   * convenient than submitting `Callable` objects directly.
   */
  def fromExecutorService(es: ExecutorService): Strategy = new Strategy {
    def apply[A](a: => A): () => A = {
      val f = es.submit { new Callable[A] { def call = a} }
      () => f.get
    }
  }

  /**
   * A `Strategy` which begins executing its argument immediately in the calling thread.
   */
  def sequential: Strategy = new Strategy {
    def apply[A](a: => A): () => A = {
      val r = a
      () => r
    }
  }
}

$line46.$read$$iw$$iw$Strategy$@591a1834

In [26]:
val echoer = Actor[String](xpool7) {
    msg => println(s"Got message: $msg")
}

$line46.$read$$iw$$iw$Actor@1db68823

In [27]:
echoer ! "my message"

Got message: my message

null

In [28]:
echoer ! "another msg"

Got message: another msg

null

In [36]:
sealed trait Future[+A] {
  def apply(k: A => Unit): Unit
}
type Par[+A] = ExecutorService => Future[A]

<console>: 14

In [30]:
def run[A](es : ExecutorService)(p: Par[A]) : A = {
    val ref = new AtomicReference[A]
    val latch = new CountDownLatch(1)
    p(es) { a => ref.set(a); latch.countDown }
    latch.await
    ref.get
}

def unit[A](a : A) : Par[A] = 
    es => new Future[A] {
        def apply(cb: A => Unit) : Unit = cb(a)
    }

def fork[A](a: => Par[A]): Par[A] = es => new Future[A] {
    def apply(cb: A => Unit): Unit = eval(es)(a(es)(cb))
}

def lazyUnit[A](a: => A): Par[A] = fork(unit(a))

def asyncF[A,B](f: A => B): A => Par[B] = a => lazyUnit(f(a))

def eval(es: ExecutorService)(r: => Unit): Unit =
  es.submit(new Callable[Unit] { def call = r })

<console>: 85

In [31]:
def map2[A,B,C](p: Par[A], p2: Par[B])(f: (A,B) => C): Par[C] = es => new Future[C] {
  def apply(cb: C => Unit): Unit = {
    var ar: Option[A] = None
    var br: Option[B] = None
    
    val combiner = Actor[Either[A, B]](es) {
        case Left(a) => br match {
            case None => ar = Some(a)
            case Some(b) => eval(es)(cb(f(a, b)))
        }
        
        case Right(b) => ar match {
            case None => br = Some(b)
            case Some(a) => eval(es)(cb(f(a, b)))
        }
    }
      
    p(es)(a => combiner ! Left(a))
    p2(es)(b => combiner ! Right(b))
  }
}

<console>: 92

In [32]:
def parMap[A,B](ps: List[A])(f: A => B): Par[List[B]] = fork { 
    val fbs: List[Par[B]] = ps.map(asyncF(f))
    sequence(fbs)
}

def sequence[A](ps: List[Par[A]]): Par[List[A]] = ps match {
    case Nil => unit(List().asInstanceOf[List[A]])
    case p :: pr   => map2(p, sequence(pr))((a1: A, a2 : List[A]) => List(a1) ++ a2)
}

<console>: 84

In [33]:
val p = parMap(List.range(1, 1000))(math.sqrt(_))

<function1>

In [34]:
run(Executors.newFixedThreadPool(2))(p).take(10)

<console>: 83

In [35]:
// id : ex_7.11
def choiceN[A](n: Par[Int])(choices: List[Par[A]]): Par[A] =
    es => choices(run(es)(n))(es)

<console>: 84